# LLM Iteration 1

**Aim**

Produce a simple initial model that generates patient arrivals for the four different patient types Stroke, TIA, Complex Neurological or Other. These will be given a unique identifier and then leave the model. 

**Prompt**

Code a discrete-event simulation model of a stroke ward in a hospital in python 3.10 and simpy 4.
Please code the full model specified. Do not return a simplified version. Show all code. 

All time units in the model are in days.
Each patient in the model has a unique identifier. The first patient to arrive has an identifier of 0. For each subsequent patient increment the identifier by 1.

Patients to arrive in a stroke ward that admits four different patient types: Stroke, TIA, Complex Neurological or Other. Each patient type had a different interarrival time distribution. After patients arrive, they immediately leave the model. The model has no capacity limits so daily occupancy is Poisson distributed. 

All patient types must have their own generator function. 

The model should print out useful information after each event. 
The model should include a user settable run length. This defaults to 2 months. 

The interarrival time distribution and parameters of patients are dependent on the patient type. This describes the average time between required admissions. For each distribution time is measured in days. 
Stroke = Exponential: 1.2
TIA = Exponential: 9.3
Complex Neurological = Exponential: 3.6
Other = Exponential: 3.2

Numpy should be used for sampling. Each interarrival distribution should have its own numpy.random.Generator object. 

Provide sources.

**LLM Output**

Sources:
1. SimPy Documentation: https://simpy.readthedocs.io/en/latest/
2. NumPy Documentation: https://numpy.org/doc/stable/

In [3]:
import simpy
import numpy as np

In [4]:
class StrokeWard:
    def __init__(self, env, num_beds):
        self.env = env
        self.num_beds = num_beds
        self.patients = {}
        self.patient_id = 0
        self.interarrival_times = {
            'Stroke': 1.2,
            'TIA': 9.3,
            'Complex Neurological': 3.6,
            'Other': 3.2
        }
        self.generators = {patient_type: np.random.default_rng() for \
                           patient_type in self.interarrival_times}

    def patient_generator(self, patient_type):
        while True:
            interarrival_time = self.generators[patient_type].\
                                exponential(self.\
                                            interarrival_times[patient_type])
            yield self.env.timeout(interarrival_time)
            self.patient_id += 1
            print(f'Patient {self.patient_id} of type {patient_type} arrives '
                  f'at day {round(self.env.now, 2)}')

    def run_simulation(self, run_length=60):
        for patient_type in self.interarrival_times:
            self.env.process(self.patient_generator(patient_type))

        while self.env.now < run_length:
            yield self.env.timeout(1)

In [5]:
# Setup and start the simulation
env = simpy.Environment()
ward = StrokeWard(env, num_beds=4)
env.process(ward.run_simulation())
env.run(until=60)  # Set the simulation run length to 60 days

Patient 1 of type Stroke arrives at day 0.5
Patient 2 of type Other arrives at day 0.79
Patient 3 of type Other arrives at day 3.43
Patient 4 of type Complex Neurological arrives at day 3.52
Patient 5 of type TIA arrives at day 3.68
Patient 6 of type Other arrives at day 5.0
Patient 7 of type Other arrives at day 5.7
Patient 8 of type Complex Neurological arrives at day 5.77
Patient 9 of type Stroke arrives at day 6.5
Patient 10 of type Other arrives at day 6.66
Patient 11 of type Stroke arrives at day 7.1
Patient 12 of type Stroke arrives at day 7.16
Patient 13 of type Stroke arrives at day 7.33
Patient 14 of type Complex Neurological arrives at day 9.67
Patient 15 of type TIA arrives at day 9.74
Patient 16 of type Other arrives at day 9.84
Patient 17 of type Stroke arrives at day 10.42
Patient 18 of type Stroke arrives at day 10.44
Patient 19 of type Complex Neurological arrives at day 10.97
Patient 20 of type Stroke arrives at day 11.53
Patient 21 of type TIA arrives at day 11.72
Pa

**Checks**

In [1]:
class TestCase():

    def __init__(self):
        self.env = simpy.Environment()
        self.ward = StrokeWard(self.env, num_beds=4)
        self.run_length = 10

    def test_run_length(self):
        '''
        Checks that specified run length is implemented correctly.
        '''
        self.env.process(self.ward.run_simulation())
        self.env.run(until=self.run_length)

        if self.env.now > self.run_length:
            print('The simulation has continued for longer than input.')

In [28]:
test = TestCase()
test.test_run_length() # Change length of time simulation has run

Patient 1 of type Complex Neurological arrives at day 0.47
Patient 2 of type Stroke arrives at day 0.6
Patient 3 of type TIA arrives at day 0.89
Patient 4 of type Other arrives at day 1.26
Patient 5 of type Other arrives at day 1.75
Patient 6 of type Other arrives at day 1.91
Patient 7 of type Complex Neurological arrives at day 2.98
Patient 8 of type Stroke arrives at day 3.63
Patient 9 of type Other arrives at day 5.41
Patient 10 of type Complex Neurological arrives at day 5.61
Patient 11 of type Other arrives at day 5.64
Patient 12 of type Stroke arrives at day 5.74
Patient 13 of type Stroke arrives at day 6.45
Patient 14 of type Other arrives at day 6.79
Patient 15 of type Stroke arrives at day 6.85
Patient 16 of type Other arrives at day 7.55
Patient 17 of type Stroke arrives at day 7.98
Patient 18 of type Complex Neurological arrives at day 8.77


**Reflection**

The LLM created patient arrivals that incremented the patient ID each time and arrived up to the specified time. The different patient types entering the model were chosen randomly using a random number generator. This meant that only one patient of a certain type could arrive at one time during the course of the model run, as opposed to them arriving independently. 

It did not use the default run time inside the run_simulation() method as it is set in env.run(until=). If this is excluded, it runs in an infinite loop. It also produces different results each time the code is run so there is no option for reproducibility. At this point, the number of beds does not have an impact as the patients leave the model as soon as they arrive so there is no reason to test this part of the model. 